In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import sys

sys.path.append('/content/drive/My Drive/ComputerVisionProject')

In [ ]:
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
import cv2
import numpy as np


def get_frames(path):
    vid = cv2.VideoCapture(path)
    
    frames = []
    
    count = 0
    while True:

      if count % 3 == 0:
        ret, frame = vid.read()
    
        if ret == True:
          if (frame.shape[0] >= 559 and frame.shape[1] >= 859):
            frame = frame[142:558,442:858] 
            frames.append(frame)
        else:
          break
      
      count += 1
    
    vid.release()
    return np.array(frames)

  

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(416, 416, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((416, 416, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


In [ ]:
def prepare_all_videos(path):
    
    video_paths = os.listdir(path)
    num_samples = len(video_paths)
    for i in range(num_samples):
      video_paths[i] = path + video_paths[i]  

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_features = np.zeros(
        shape=(num_samples, 350, 2048), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print("On pitch: {}".format(idx))
        # Gather all its frames and add a batch dimension.
        frames = get_frames(path)
        frames = frames[None, ...]
        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, 350, 2048), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(video_length, 350)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features

In [ ]:
fastballs = prepare_all_videos("/content/drive/My Drive/ComputerVisionProject/FF/")

In [ ]:
curveballs = prepare_all_videos("/content/drive/My Drive/ComputerVisionProject/CU (1)/")

In [ ]:
print(fastballs.shape)
print(curveballs.shape)

(201, 350, 2048)
(291, 350, 2048)


In [ ]:
# Utility for our sequence model.
def get_sequence_model():

    frame_features_input = keras.Input((350, 2048))
    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(2, activation="softmax")(x)

    rnn_model = keras.Model(frame_features_input, output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

In [ ]:


def run_experiment():

    fastball_labels = np.ones(fastballs.shape[0])
    curveball_labels = np.zeros(curveballs.shape[0])
    labels = np.concatenate((fastball_labels[0 : 200], curveball_labels[0 : 200]))
    pitches = np.concatenate((fastballs[0 : 200], curveballs[0 : 200]))

    idx = np.random.permutation(len(pitches))
    X, Y = pitches[idx], labels[idx]


    X_train = X[0 : 150]
    Y_train = Y[0 : 150]

    X_test = X[150 : ]
    Y_test = Y[150 : ]

    print(sum(Y_test) / len(Y_test))
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        X_train,
        Y_train,
        validation_split=0.3,
        epochs=25,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate(X_test, Y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model, accuracy

In [ ]:
total_accuracy = 0
for i in range(50):
  _, sequence_model, accuracy = run_experiment()
  total_accuracy += accuracy
print(total_accuracy / 25)

Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 77ms/step - loss: 0.5892 - accuracy: 0.6952 - val_loss: 0.6798 - val_accuracy: 0.4222
Epoch 13/25
3/4 [=====================>........] - ETA: 0s - loss: 0.6067 - accuracy: 0.6979
Epoch 13: val_loss did not improve from 0.67978
4/4 [==============================] - 0s 70ms/step - loss: 0.6071 - accuracy: 0.6857 - val_loss: 0.6804 - val_accuracy: 0.4222
Epoch 14/25
3/4 [=====================>........] - ETA: 0s - loss: 0.5752 - accuracy: 0.7292
Epoch 14: val_loss improved from 0.67978 to 0.67974, saving model to /tmp/video_classifier
4/4 [==============================] - 0s 77ms/step - loss: 0.5811 - accuracy: 0.7143 - val_loss: 0.6797 - val_accuracy: 0.4222
Epoch 15/25
3/4 [=====================>........] - ETA: 0s - loss: 0.5600 - accuracy: 0.7292
Epoch 15: val_loss improved from 0.67974 to 0.67537, saving model to /tmp/video_classifier
4/4 [==============================] - 0s 78ms/step - lo

In [ ]:
print(total_accuracy/50)

0.7003200030326844


In [ ]:
sequence_model.save("/content/drive/My Drive/ComputerVisionProject/Model")